In [ ]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')

from PatchyConverter import PatchyConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

### Load data

In [2]:
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
relabeling = True

In [3]:
graph_converter = PatchyConverter(dataset_name,receptive_field)

importing graph data
number of graphs in MUTAG dataset : 188
number of features : 7
Separating Graphs per graph ID
getting width
width: 18


In [5]:
graph_tensor = graph_converter.graphs_to_Patchy_tensor()

MUTAG tensor exists, loading it from Dropbox
Loading path: /Users/marcelogutierrez/.gamma_link/Samples/MUTAG/MUTAG_patchy_tensor_w_18.npy


In [6]:
# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)


In [7]:
# Capsule Architecture Parameters:
capsule_params = CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params, conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2, 2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params, caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params, digit_layer_name)

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256  # 250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params, decoder_layer)

# Training Hyperparameters:

args_train = CapsuleTrainingParameters()

In [13]:
data_split = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state =0)
x_train, x_test, y_train, y_test = data_split
data = ((x_train, y_train), (x_test, y_test))
input_shape = x_train.shape[1:]
n_class = len(np.unique(y_train))

In [14]:
patchy_classifier = GraphClassifier(input_shape,n_class)
patchy_classifier.build_the_graph(capsule_params)

Instructions for updating:
dim is deprecated, use axis instead
time to generate the model: 0.6146979331970215


(<keras.engine.training.Model at 0x1a308efeb8>,
 <keras.engine.training.Model at 0x1a30930c50>)

In [15]:
patchy_classifier.train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 18, 10, 7)    0                                            
__________________________________________________________________________________________________
relu (Conv2D)                   (None, 10, 2, 256)   145408      input_1[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 5, 1, 256)    262400      relu[0][0]                       
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 160, 8)       0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [16]:
patchy_classifier.train_model.losses

[]

In [21]:
for layer in patchy_classifier.train_model.layers:
    weights = layer.get_weights() 
    for weight in weights:
        print(weight.shape)

(9, 9, 7, 256)
(256,)
(2, 2, 256, 256)
(256,)
(2, 160, 16, 8)
(32, 256)
(256,)
(256, 512)
(512,)
(512, 1260)
(1260,)
